In [1]:
import pandas as pd
from numpy import *

#read excel files
file = pd.ExcelFile("~/Desktop/Personal Financial Example_ (3).xlsx")
Sheet_Names = file.sheet_names[1:41]
print("Sheet Names:",Sheet_Names)

#read excel sheets into a map
df = pd.read_excel("~/Desktop/Personal Financial Example_ (3).xlsx",sheet_name=None)

Sheet Names: ['Sheet1', 'Sheet2', 'Sheet3', 'Sheet4', 'Sheet5', 'Sheet6', 'Sheet7', 'Sheet8', 'Sheet9', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet13', 'Sheet14', 'Sheet15', 'Sheet16', 'Sheet17', 'Sheet18', 'Sheet19', 'Sheet20', 'Sheet21', 'Sheet22', 'Sheet23', 'Sheet24', 'Sheet25', 'Sheet26', 'Sheet27', 'Sheet28', 'Sheet29', 'Sheet30', 'Sheet31', 'Sheet32', 'Sheet33', 'Sheet34', 'Sheet35', 'Sheet36', 'Sheet37', 'Sheet38', 'Sheet39', 'Sheet40']


In [2]:
#a sample data
df['Sheet2'].head(20)

#data types
df['Sheet2'].dtypes

TRANSNumber             int64
TimeStamp      datetime64[ns]
DA                    float64
Oper                    int64
CTR                    object
dtype: object

In [3]:
#all column headers
df['Sheet2'].columns

Index(['TRANSNumber', 'TimeStamp', 'DA', 'Oper', 'CTR'], dtype='object')

In [4]:
#correct errors of Sheet5: missing negative signs of outcome
df5=df['Sheet5']
df5_ctr = df5['CTR'].str.split('_',expand=True)
df5 = pd.concat([df5, df5_ctr[1]], axis=1, sort=False)
df5 = df5.reset_index()

cat_name=pd.Series(df5[1]).unique()
for cat in cat_name:
    index = df5.index[df5[1] == cat].tolist()
    for i in index:
        df5=df5.replace(df5['DA'][i],(df5['DA'][i])*(-1))
df['Sheet5']=df5

In [6]:
#define a function that calculate portions of outcome for each CTR categorical

def analyze_significance(df,name):
    data = df[name][['DA','CTR']].loc[df[name]['DA']<0]   
    data_ctr = data['CTR'].str.split('_',expand=True) #split 3 kinds CTR alone
    data = pd.concat([data, data_ctr[1]], axis=1, sort=False)
    data = data.reset_index()
    cat_name=pd.Series(data[1]).unique() #3 types CTR
    d=[]
    p=[]
    d_p=[]
    for cat in cat_name:
        index = data.index[data[1] == cat].tolist()
        num=0
        for i in index:
            num+=data['DA'][i]
            avg=mean(data['DA'])
            percent=num/sum(data['DA'])
        p+=[percent]
        p_mean=mean(p)
        d+=[cat,percent]
        d_p=list(sort(d[1::2]))   #sort the percent
    return d,p_mean,d_p

In [6]:
#apply defined function to all sheets and doing the sorting
l=[]
p_mean_l=[]
d_p_l=[]
for name in Sheet_Names:
    d,p_mean,d_p=analyze_significance(df,name)
    l+=d
    p_mean_l+=[p_mean,p_mean,p_mean]
    d_p_l+=d_p
    
#create a table containing portions of 3 labels from 40 sheets
names=[]
for name in Sheet_Names:
    names+=3*[name]
    
table=pd.DataFrame({"Sheet Name":names,"CTR":l[::2],"Percent":l[1::2]})

#display table
table

,Sheet Name,CTR,Percent
0,Sheet1,Superfluous,0.335468
1,Sheet1,Investment,0.311184
2,Sheet1,Essencials,0.353348
3,Sheet2,Superfluous,0.248140
4,Sheet2,Investment,0.309835
5,Sheet2,Essencials,0.442025
6,Sheet3,Superfluous,0.413282
7,Sheet3,Essencials,0.325852
8,Sheet3,Investment,0.260866
9,Sheet4,Essencials,0.360730


In [7]:
#create new columns that is percent minus average
table_percent=reshape(array(table['Percent']),(len(table),1))
new_percent=pd.DataFrame((table_percent-array([p_mean_l]).T)*100)  
table['Percent Difference']=new_percent
table.head(9)

,Sheet Name,CTR,Percent,Percent Difference
0,Sheet1,Superfluous,0.335468,0.213488
1,Sheet1,Investment,0.311184,-2.214908
2,Sheet1,Essencials,0.353348,2.001421
3,Sheet2,Superfluous,0.248140,-8.519298
4,Sheet2,Investment,0.309835,-2.349822
5,Sheet2,Essencials,0.442025,10.869121
6,Sheet3,Superfluous,0.413282,7.994848
7,Sheet3,Essencials,0.325852,-0.748147
8,Sheet3,Investment,0.260866,-7.246701


In [8]:
#doing the sorting process
ctr_max=[]
ctr_min=[]

#return labels of maximun/minimum percent for each sheet
for i in range(118):
    temp=table.iloc[i:i+3]
    temp=(temp.sort_values(by=['Percent']))
    i+=4
    ctr_max+=[array(temp)[2][1]]
    ctr_min+=[array(temp)[0][1]]
ctr_max=ctr_max[slice(0,120,3)]
ctr_min=ctr_min[slice(0,120,3)]

print('max_label:',ctr_max)
#print('min_label:',ctr_min)

max_label: ['Essencials', 'Essencials', 'Superfluous', 'Essencials', 'Essencials', 'Essencials', 'Superfluous', 'Essencials', 'Investment', 'Investment', 'Superfluous', 'Essencials', 'Investment', 'Investment', 'Superfluous', 'Investment', 'Essencials', 'Essencials', 'Superfluous', 'Superfluous', 'Essencials', 'Investment', 'Investment', 'Essencials', 'Essencials', 'Superfluous', 'Investment', 'Investment', 'Essencials', 'Investment', 'Investment', 'Essencials', 'Investment', 'Superfluous', 'Superfluous', 'Superfluous', 'Essencials', 'Investment', 'Superfluous', 'Investment']


In [9]:
#sorting between these 40 max_labels
num_essencials=0
num_investment=0
for i in ctr_max:
    if i=='Essencials':
        num_essencials+=1

for i in ctr_max:
    if i=='Investment':
        num_investment+=1

print("Essencials:",num_essencials/40,"Investment",num_investment/40,"Superfluous",1-num_investment/40-num_essencials/40)

Essencials: 0.375 Investment 0.35 Superfluous 0.275


In [10]:
#sorting between these 40 min_labels
num_essencials_1=0
num_investment_1=0
for i in ctr_min:
    if i=='Essencials':
        num_essencials_1+=1

for i in ctr_max:
    if i=='Investment':
        num_investment_1+=1

print("Essencials:",num_essencials_1/40,"Investment",num_investment_1/40,"Superfluous",1-num_investment_1/40-num_essencials_1/40)

Essencials: 0.25 Investment 0.35 Superfluous 0.4
